In [ ]:
import requests
import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver

In [ ]:
# selenium을 백그라운드로 실행하기 위해 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('window-size=1920x1080')

In [ ]:
# 우리가 데이터를 가져올 웹 페이지입니다.
faker_opgg_url = 'https://www.op.gg/summoner/userName=Hide%20on%20bush'

# selenium이 제어할 chrome을 실행합니다.
driver = webdriver.Chrome("/Users/namshin/chromedriver")
# driver = webdriver.Chrome("/Users/namshin/chromedriver", options= options)

In [ ]:
# 데이터를 가져올 페이지로 이동합니다.
driver.get(faker_opgg_url)

In [ ]:
# 챔피언 검색창에 'Syndra' 입력
driver.find_element_by_id('right_search_champion').find_element_by_css_selector('input.Input').send_keys('Syndra\n')
time.sleep(1)

In [ ]:
# 매치 탭에서 'Ranked Solo' 클릭
driver.find_element_by_id('right_gametype_soloranked').click()
time.sleep(1)

In [ ]:
# '더 보기' 버튼이 활성화되어있으면(리스트의 끝까지 도달하지 않았으면) 계속 클릭하여 이전 게임을 모두 로딩합니다.
while True:
    try:
        driver.find_element_by_css_selector('.GameMoreButton').click()
        # 게임 로딩, html 코드 변경까지 여유 시간을 1초 가집니다.
        time.sleep(1)
        
    # 에러가 나면(페이지에서 '더 보기' 버튼이 없을 경우) while문을 탈출합니다.
    except Exception as e:
        pass
        break

In [ ]:
# 결과가 들어갈 빈 리스트를 만듭니다.
faker_champions = []
faker_kills = []
faker_deaths = []
faker_assists = []
faker_results = []

In [ ]:
# 각 게임에 대해 웹 페이지에 기재된 스탯을 찾아서(selector 사용) 결과 리스트에 append하기
for gamelist in driver.find_elements_by_css_selector('div.GameItemList'):
    for game in gamelist.find_elements_by_css_selector('div.GameItemWrap'):
        faker_champions.append(game.find_element_by_css_selector('div.ChampionName').text.strip())
        faker_kills.append(game.find_element_by_css_selector('div.KDA').
                           find_element_by_css_selector('div.KDA').
                           find_element_by_css_selector('span.Kill').text.strip())
        faker_deaths.append(game.find_element_by_css_selector('div.KDA').
                           find_element_by_css_selector('div.KDA').
                           find_element_by_css_selector('span.Death').text.strip())
        faker_assists.append(game.find_element_by_css_selector('div.KDA').
                           find_element_by_css_selector('div.KDA').
                           find_element_by_css_selector('span.Assist').text.strip())
        faker_results.append(game.find_element_by_css_selector('div.GameStats').
                           find_element_by_css_selector('div.GameResult').text.strip())

In [ ]:
# selenium이 제어하는 크롬을 종료합니다.
driver.quit()

In [ ]:
# DataFrame으로 변환 후 출력
faker_total_df = pd.DataFrame([faker_champions,
                                faker_results,
                                faker_kills,
                                faker_deaths,
                                faker_assists],
                               index = ['champion', 'result', 'kills', 'deaths', 'assists']).T
faker_total_df